In [2]:
import sklearn
import numpy as np
import pandas as pd

In [18]:
# import data
train = pd.read_csv('../data/raw/Emergent_NAACL2016/emergent/url-versions-2015-06-14-clean-train.csv')
train.drop(columns=["Unnamed: 0"], inplace=True)
test = pd.read_csv('../data/raw/Emergent_NAACL2016/emergent/url-versions-2015-06-14-clean-test.csv')
test.drop(columns=["Unnamed: 0"], inplace=True)

In [5]:
align_feature = pd.read_csv('../data/processed/features/alignment_feature.csv')
cosine_feature = pd.read_csv('../data/processed/features/cosine_feature.csv')
bow_feature = pd.read_csv('../data/processed/features/headline_BoWF.csv')
qmark_feature = pd.read_csv('../data/processed/features/headline_Qmark.csv')
neg_alignment_feature = pd.read_csv('../data/processed/features/neg_alignment_feature.csv')
root_dist_feature = pd.read_csv('../data/processed/features/root_dist.csv')
svo_feature = pd.read_csv('../data/processed/features/svo.csv')

In [20]:
target_map = {"for": 0, "observing": 1, "against": 2}

In [21]:
train = train.replace({"articleHeadlineStance": target_map})

In [22]:
test = test.replace({"articleHeadlineStance": target_map})

In [23]:
train["articleHeadlineStance"].value_counts()

0    992
1    775
2    304
Name: articleHeadlineStance, dtype: int64

In [24]:
train = pd.merge(train, align_feature, on="articleId")
train = pd.merge(train, cosine_feature, on="articleId")
train = pd.merge(train, bow_feature, on="articleId")
train = pd.merge(train, qmark_feature, on="articleId")
train = pd.merge(train, neg_alignment_feature, on="articleId")
train = pd.merge(train, root_dist_feature, on="articleId")
train = pd.merge(train, svo_feature, on="articleId")

In [25]:
test = pd.merge(test, align_feature, on="articleId")
test = pd.merge(test, cosine_feature, on="articleId")
test = pd.merge(test, bow_feature, on="articleId")
test = pd.merge(test, qmark_feature, on="articleId")
test = pd.merge(test, neg_alignment_feature, on="articleId")
test = pd.merge(test, root_dist_feature, on="articleId")
test = pd.merge(test, svo_feature, on="articleId")

In [26]:
train["articleHeadlineStance"] = train["articleHeadlineStance"].astype("int")

In [27]:
test["articleHeadlineStance"] = test["articleHeadlineStance"].astype("int")

In [28]:
# merge datasets and shuuffle with random seed
seed = 1234

train = sklearn.utils.shuffle(train, random_state=seed)
test = sklearn.utils.shuffle(test, random_state=seed)


In [29]:
train

,claimHeadline,articleHeadline,articleHeadlineStance,articleId,claimId,alignmentScore,cosineSim,1,2,3,...,20551,Qmark,negAlignmentScore,Unnamed: 0,rootDistance,hedgedRootDistance,refuteRootDistance,subject_entailments,object_entailments,relation_entailments
1605,A Catholic priest who was declared clinically ...,Op-Ed: Hoax Priest who never existed claims he...,2,8fbd8440-bb74-11e4-b073-ed15c88bfa1e,f5891b80-ba9e-11e4-b073-ed15c88bfa1e,-4.285714,-0.049733,0,0,0,...,0,0,1,2007,5,6,0,2,0,0
973,ISIS beheaded journalist James Wright Foley,Islamic State releases video apparently showin...,1,a76737d0-2803-11e4-a916-bf0b27b73ad5,c38db880-27e0-11e4-96be-dfc0d1eebaea,3.333333,-0.763106,0,0,0,...,0,0,0,1265,0,0,0,2,0,0
484,An abandoned dog was found at Scottish railway...,Dog abandoned at Ayr station with belongings i...,0,63254080-9685-11e4-87a8-1d5d45e62193,26ced990-9683-11e4-87a8-1d5d45e62193,5.555556,0.008224,0,0,0,...,0,0,0,666,0,0,0,0,0,0
344,A man was saved from a bear attack thanks to a...,"Finally, proof that Justin Bieber IS unbearabl...",0,641ae450-1ce5-11e4-8c09-cd48fe37674e,27b04730-1ce5-11e4-8c09-cd48fe37674e,0.000000,0.001697,0,0,0,...,0,0,0,452,0,0,0,2,0,0
793,The iPhone 6 pulls out facial hair when being ...,Is the iPhone 6 Really Pulling Out Your Hair?,1,1694cb60-4e5e-11e4-949b-f14041b20838,ce44add0-4e5d-11e4-949b-f14041b20838,3.333333,0.377893,0,0,0,...,0,1,0,1039,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,Abdel-Majed Abdel Bary beheaded James Wright F...,"The rapper, the hacker and the terror travel g...",1,54860a50-2bf5-11e4-89c5-1d389b94bc4b,98933d20-2bec-11e4-89c5-1d389b94bc4b,-1.428571,0.002429,0,0,0,...,0,1,0,387,0,0,0,2,0,0
2041,Lovesick woman stays in KFC for one week,Heartbroken girl spends week in KFC after gett...,0,b72e5410-9ff8-11e4-a353-5b831fe927be,40f8caf0-9ff8-11e4-a353-5b831fe927be,-2.500000,0.011158,0,0,0,...,0,0,0,2565,0,0,0,2,0,0
664,The group behind the Sony hack sent a message ...,"Hackers tell Sony wise to cancel film, don't m...",1,2f93ed80-8798-11e4-a13d-d9bce527941d,9e769320-8797-11e4-a13d-d9bce527941d,-7.142857,-0.000153,0,0,0,...,0,0,1,868,8,0,0,2,0,0
1318,Video shows a U.S. Marine surviving being shot...,US Marine survives headshot in Afghanistan,0,1e61e760-5543-11e4-b6a0-9705a8f94440,74b32120-553d-11e4-b6a0-9705a8f94440,-3.333333,-0.231769,0,0,0,...,0,0,0,1639,0,0,0,2,0,0


In [30]:
# create train and test

X_train = train.to_numpy()[:,5:]
Y_train = train["articleHeadlineStance"].values.reshape((-1,))

X_test = test.to_numpy()[:,5:]
Y_test = test["articleHeadlineStance"].values.reshape((-1,))

In [31]:
Y_train = Y_train.astype(int)

In [32]:
X_train = X_train.astype(float)

In [33]:
X_test = X_test.astype(float)

In [34]:
y_test = Y_test.astype(int)

In [35]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(2071, 20562)
(2071,)
(524, 20562)
(524,)


In [36]:
from sklearn.model_selection import StratifiedKFold

In [37]:
train["articleHeadlineStance"].value_counts()

0    992
1    775
2    304
Name: articleHeadlineStance, dtype: int64

In [38]:
train["articleHeadlineStance"].value_counts(normalize=True) * 100

0    47.899565
1    37.421535
2    14.678899
Name: articleHeadlineStance, dtype: float64

In [39]:
test['articleHeadlineStance'].value_counts(normalize=True) * 100

0    46.946565
1    35.687023
2    17.366412
Name: articleHeadlineStance, dtype: float64

In [40]:
skf = StratifiedKFold(n_splits=5, )

In [41]:
skf.get_n_splits(X_train, Y_train)

5

In [42]:
train_folds = []
valid_folds = []
for i, (train_index, test_index) in enumerate(skf.split(X_train, Y_train)):
    print("TRAIN:", train_index.shape, "TEST:", test_index.shape)
    x_train_fold, x_valid_fold = X_train[train_index], X_train[test_index]
    y_train_fold, y_valid_fold = Y_train[train_index], Y_train[test_index]
    bc = np.bincount(y_train_fold)
    ii = np.nonzero(bc)[0]
    print("train set distribution")
    print(list(zip(ii,bc[ii])))
    bc = np.bincount(y_valid_fold)
    ii = np.nonzero(bc)[0]
    print("validation set distribution")
    print(list(zip(ii,bc[ii])))
    print("----------------------------")
    train_fold = np.concatenate((x_train_fold, y_train_fold.reshape(-1, 1)), axis=1)
    valid_fold = np.concatenate((x_valid_fold, y_valid_fold.reshape(-1, 1)), axis=1)
    train_folds.append(train_fold)
    valid_folds.append(valid_fold)

TRAIN: (1656,) TEST: (415,)
train set distribution
[(0, 793), (1, 620), (2, 243)]
validation set distribution
[(0, 199), (1, 155), (2, 61)]
----------------------------
TRAIN: (1656,) TEST: (415,)
train set distribution
[(0, 793), (1, 620), (2, 243)]
validation set distribution
[(0, 199), (1, 155), (2, 61)]
----------------------------
TRAIN: (1657,) TEST: (414,)
train set distribution
[(0, 794), (1, 620), (2, 243)]
validation set distribution
[(0, 198), (1, 155), (2, 61)]
----------------------------
TRAIN: (1657,) TEST: (414,)
train set distribution
[(0, 794), (1, 620), (2, 243)]
validation set distribution
[(0, 198), (1, 155), (2, 61)]
----------------------------
TRAIN: (1658,) TEST: (413,)
train set distribution
[(0, 794), (1, 620), (2, 244)]
validation set distribution
[(0, 198), (1, 155), (2, 60)]
----------------------------


In [44]:
np.savez("../data/processed/train/train_folds.npz", *train_folds)

In [46]:
np.savez("../data/processed/valid/valid_folds.npz", *valid_folds)

In [46]:
folds = np.load('../data/processed/valid/valid_folds.npz')

In [64]:
for k in folds:
    valid_x = folds[k][:, :-1] \
    valid_y = folds[k][:, -1:]
    print(valid_x.shape)
    print(valid_y.shape)

(415, 20552)
(415, 1)
(414, 20552)
(414, 1)
(414, 20552)
(414, 1)
(414, 20552)
(414, 1)
(414, 20552)
(414, 1)
